In [6]:
import os
import time
import gnupg
import keyring
from binance.spot import Spot
import numpy as np
import pandas as pd
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
import gymnasium as gym
from gymnasium import spaces

In [2]:
gpg = gnupg.GPG()
encrypted_file_path = os.path.expanduser('~/Repositorios/binance_trader/.env.gpg')
with open(encrypted_file_path, 'rb') as file:
    datos = gpg.decrypt_file(file, passphrase=keyring.get_password("GPG_Passphrase", "gpg_python"))
if datos.ok:
    env_vars = dict(line.decode('utf-8').split('=', 1) for line in datos.data.splitlines())
    api_key = env_vars.get('API_KEY')
    secret_key = env_vars.get('SECRET_KEY')
# Inicializa el cliente de Binance
client = Spot(api_key, secret_key)

In [3]:
# Obtén información sobre el intercambio
exchange_info = client.exchange_info()
# Filtra los pares de trading contra USDT
usdc_pairs = [symbol['symbol'] for symbol in exchange_info['symbols'] if symbol['quoteAsset'] == 'USDC']

In [14]:
def get_historical_data(symbol, interval='1d'):
    start_time = 0
    data = []
    while True:
        # Obtener datos de klines
        klines = client.klines(symbol=symbol, interval=interval, limit=1000, startTime=start_time)
        # Si no hay más datos, salir del bucle
        if not klines:
            break
        for kline in klines:
            data.append({
                'open_time': kline[0],
                'open': float(kline[1]),
                'high': float(kline[2]),
                'low': float(kline[3]),
                'close': float(kline[4]),
                'volume': float(kline[5]),
                'quote_asset_volume': float(kline[7]),
                'number_of_trades': kline[8],
                'taker_buy_base_asset_volume': float(kline[9]),
                'taker_buy_quote_asset_volume': float(kline[10])
            })
        # Actualizar start_time para la próxima solicitud
        start_time = klines[-1][0] + 1
    return pd.DataFrame(data)

In [15]:
btcusdc_data = get_historical_data(usdc_pairs[1])
btcusdc_data.set_index('open_time', inplace=True)
btcusdc_data.index = pd.to_datetime(btcusdc_data.index, unit='ms').to_period('D')
btcusdc_data.tail()

0
1631145600001
1717804800001


,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
open_time,,,,,,,,,
2024-06-04,68796.00,71080.00,68549.54,70544.37,1226.98052,8.608864e+07,205914,559.56689,3.922868e+07
2024-06-05,70544.37,71771.68,70384.02,71110.36,1522.32085,1.081511e+08,63124,763.78407,5.424188e+07
2024-06-06,71114.94,71643.99,70100.81,70771.35,970.92274,6.892657e+07,56753,456.66717,3.242022e+07
2024-06-07,70771.36,71990.00,68368.00,69322.37,1462.02731,1.030713e+08,134021,771.65816,5.438884e+07
2024-06-08,69322.36,69552.00,69200.01,69468.46,132.46129,9.190252e+06,19619,73.99440,5.133604e+06
